# Install and Import Library

In [ ]:
! pip install grandalf langchain langchain-community langchain-groq langchain-anthropic langchain-google-genai langchain-openai langchain-huggingface

In [ ]:
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFacePipeline, HuggingFaceEmbeddings

import os

In [ ]:
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

# Model loading

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # repo_id = "perplexity-ai/r1-1776",
    task="text-generation"
  )
# llm=HuggingFaceEndpoint(repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",task="text-generation")
hf_model = ChatHuggingFace(llm=llm)

hf_model.invoke("Hi I am Lokesh")

In [ ]:
gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-pro')
gemini_model.invoke("Hi I am Lokesh")

In [ ]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")
groq_model.invoke("Hi I am Lokesh")

In [ ]:
open_ai_model = ChatOpenAI(model="gpt-4o")
open_ai_model.invoke("Hi I am Sunny")

In [ ]:
claude_model = ChatAnthropic(model="claude-2")
claude_model.invoke("Hi I am Sunny")

In [ ]:
llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    pipeline_kwargs={
        "temperature": 0.5,
        "max_new_tokens":100
        }
)

model = ChatHuggingFace(llm=llm)
model.invoke("Hi I am Lokesh")

# Embedding Models

In [ ]:
# @markdown # OpenAI

embedding_model = "text-embedding-3-small" # @param ["text-embedding-3-large","text-embedding-3-small","text-embedding-ada-002"]
dimensions = 64 #@param {type:"integer"}
set_dimension = True # @param {type:"boolean"}
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}

if set_dimension:
  openai_embedding = OpenAIEmbeddings(
      model=embedding_model,
      dimensions=dimensions,
  )
else:
  openai_embedding = OpenAIEmbeddings(
      model=embedding_model,
  )

result = openai_embedding.embed_query(query)
print(len(result),result)

In [ ]:
# @markdown # Google GenAi

embedding_model = "models/embedding-001" # @param ["models/gemini-embedding-exp-03-07","models/text-embedding-004","models/embedding-001"]
task_type = "None" # @param ["None","task_type_unspecified","retrieval_query","retrieval_document","semantic_similarity","classification","clustering"]
transport = "None" # @param ["None","rest","grpc","grpc_asyncio"]
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}

func = lambda x : None if x=="None" else x
task_type = func(task_type)
transport = func(transport)

google_embedding = GoogleGenerativeAIEmbeddings(
    model=embedding_model,
    task_type=task_type,
    transport=transport
)

result = google_embedding.embed_query(query)
print(len(result),result)


In [ ]:
# @markdown # Hugging Face
model_name = "BAAI/bge-en-icl" # @param ["BAAI/bge-en-icl","all-MiniLM-L6-v2"]
query = "India is a growing country" # @param {"type":"string","placeholder":"India is a growing country"}
huggingface_embeddings=HuggingFaceEmbeddings(model_name=model_name)

result = huggingface_embeddings.embed_query(query)
print(len(result),result)

In [ ]:
documents=["what is a capital of USA",
           "who is a president of usa",
           "who is a prime minister of india"]

results = openai_embedding.embed_documents(documents)
print(len(results),len(results[0]))
print(results[0])

In [ ]:
results = google_embedding.embed_documents(documents)
print(len(results),len(results[0]))
print(results[0])

In [ ]:
results = huggingface_embeddings.embed_documents(documents)
print(len(results),len(results[0]))
print(results[0])

# PROMPTS or PROMPT_TEMPLATE

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [ ]:
SYSTEM_MSG = SystemMessage(
    "you are a funny bot means whatever you answer, you answer in the funny way"
)

HUMAN_MSG = HumanMessage(
    "Who is your best friend"
)

messages = [
    SYSTEM_MSG, HUMAN_MSG
]

In [ ]:
gemini_model.invoke(messages)

In [ ]:
groq_model.invoke(messages)

In [ ]:
open_ai_model.invoke(messages)

In [ ]:
claude_model.invoke(messages)

In [ ]:
model.invoke(messages)

## Prompt template

In [ ]:
template = PromptTemplate(
    template="can you say hello to {name} in 5 different language",
    input_variables=["name"]
)

template.get_prompts()

In [ ]:
template.invoke({"name":"Lokesh"})

In [ ]:
prompt = template.invoke({"name":"Lokesh"})
gemini_model.invoke(prompt).content

## Chat Prompt Template

In [ ]:
chat_template = ChatPromptTemplate(
    messages=[
        ("system","you are a helpful {domain} expert"),
        ("human","explain the {topic} in simple terms")
    ]
)

prompt = chat_template.invoke({"domain":"medical","topic":"maleria"})
prompt

In [ ]:
gemini_model.invoke(prompt).content

# Chaining using LCEL

In [ ]:
from ast import parse
from langchain_core.output_parsers import StrOutputParser
# from langchain_core.chains import LLMChain
parser = StrOutputParser()

prompt = PromptTemplate(
    template="can you give me a detail explaination of {topic}",
    input_variables=['topic']
)

In [ ]:
chain = prompt | gemini_model | parser
chain.invoke({"topic":"astrology"})

In [ ]:
chain.get_graph().print_ascii()

# Sequential llm chain

In [ ]:
prompt1=PromptTemplate(
    template="analysis the the given text carefully {text} and take the necessary data",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="summarize the given text in 2 bullet points {text}",
    input_variables=["text"]
)

text="""In November 2023, xAI began previewing Grok as a chatbot to selected people,[10] with participation in the early access program being limited to paid X Premium users.[11]

It was announced that once the bot was out of early beta, it would only be available to higher tier X Premium+ subscribers.[12]

At the time of the preview, xAI described the chatbot as "a very early beta product – the best we could do with 2 months of training" that could "improve rapidly with each passing week".[13]

On March 11, 2024, Musk posted on X that the language model would go open source within a week. Six days later, on March 17, Grok-1 was open sourced under the Apache-2.0 license.[14][15] Disclosed were the networks architecture and its weight parameters.[16]

On March 26, 2024, Musk announced that Grok would be enabled for premium subscribers, not just those on the higher-end tier, Premium+.[17]

Grok-1.5
Grok-1.5
Developer(s)	xAI
Initial release	May 15, 2024; 10 months ago
Predecessor	Grok-1.5
Successor	Grok-2
Type
Large language model
Foundation model
License	Proprietary
Website	x.ai/blog/grok-1.5
On March 29, 2024, Grok-1.5 was announced, with "improved reasoning capabilities" and a context length of 128,000 tokens.[18] Grok-1.5 was released to all X Premium users on May 15, 2024.[1]

On April 4, 2024, an update to X's "Explore" page included summaries of breaking news stories written by Grok, a task previously assigned to a human curation team.[19]

On April 12, 2024, Grok-1.5 Vision (Grok-1.5V) was announced. Grok-1.5V is able to process a wide variety of visual information, including documents, diagrams, graphs, screenshots, and photographs.[20] Grok-1.5V was never released to the public.

On May 4, 2024, Grok became available in the United Kingdom,[21] that being the only country in Europe to support Grok at the moment due to the impending Artificial Intelligence Act rules in the European Union. Grok was later reviewed by the EU and was released on May 16, 2024.[22]"""



In [ ]:
chain = prompt1 | gemini_model | parser | prompt2 | gemini_model | parser
chain.get_graph().print_ascii()

In [ ]:
result = chain.invoke({"text":text})
print(result)

# Parallel chain

In [ ]:
prompt1=PromptTemplate(
    template="generate simple summary from the following text \n {text}",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="generate 3 question and answer from the following text \n {text}",
    input_variables=["text"]
    )

prompt3=PromptTemplate(
    template="analysis the summary and qa and generate the 5 important quiz with 4 possible answer \n summary: {summary}, Q&A: {qa}",
    input_variables=["summary","qa"]
)

In [ ]:
from langchain.schema.runnable import RunnableParallel

parallel_chain = RunnableParallel(
    summary=prompt1 | groq_model | parser,
    qa=prompt2 | groq_model | parser
)

parallel_chain

In [ ]:
parallel_chain.invoke({"text":text})

In [ ]:
chain = parallel_chain | prompt3 | groq_model | parser
chain.invoke({"text":text})

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
text="""AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, thereby exemplifying a novel form of digital agency.[1]

Intelligent agents can range from simple to highly complex. A basic thermostat or control system is considered an intelligent agent, as is a human being, or any other system that meets the same criteria—such as a firm, a state, or a biome.[2]

Intelligent agents operate based on an objective function, which encapsulates their goals. They are designed to create and execute plans that maximize the expected value of this function upon completion.[3] For example, a reinforcement learning agent has a reward function, which allows programmers to shape its desired behavior.[4] Similarly, an evolutionary algorithm's behavior is guided by a fitness function.[5]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.

Intelligent agents are often described schematically as abstract functional systems similar to computer programs. To distinguish theoretical models from real-world implementations, abstract descriptions of intelligent agents are called abstract intelligent agents. Intelligent agents are also closely related to software agents—autonomous computer programs that carry out tasks on behalf of users. They are also referred to using a term borrowed from economics: a "rational agent".[2]"""


result=chain.invoke({"text":text})


print(result)

## JSON Ouput parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_parser.get_format_instructions()

In [ ]:
template=PromptTemplate(
    template="give me name, age and city from the provided text {text} \n {format_instructions}" ,
    input_variables=['text'],
    partial_variables={"format_instructions":json_parser.get_format_instructions()}
)

text="""hi my name is Lokesh Parab my age is 25 and i am belong to mumbai"""

prompt=template.format(text=text)
prompt


result=groq_model.invoke(prompt)
result


In [ ]:
result=json_parser.parse(result.content)
result

In [ ]:
chain = template | groq_model | json_parser
chain.invoke({"text":text})

# Create Chatbot

In [ ]:
chat_history = [
    SystemMessage(
       "you are a helpful assistant"
    )
]

model_name = "openai" # @param ["openai","googleai","claudeai","groqai"]

# def get_model(model_name):
#   if model_name=="openai":
#     return ChatOpenAI("gpt-4o-mini")
#   elif model_name=="googleai":
#     return ChatGoogleGenerativeAI(model='gemini-1.5-pro')
#   elif model_name=="claudeai":
#     return ChatAnthropic(model="claude-2")
#   elif model_name=="groqai":
#     return ChatGroq(model="deepseek-r1-distill-llama-70b")
get_model={
    "openai":ChatOpenAI(model="gpt-4o-mini"),
    "googleai":ChatGoogleGenerativeAI(model='gemini-1.5-pro'),
    "claudeai":ChatAnthropic(model="claude-2"),
    "groqai":ChatGroq(model="deepseek-r1-distill-llama-70b")
}

chat_model = get_model[model_name]

while True:
  user_input = input("User_input:")

  chat_history.append(HumanMessage(content=user_input))
  if user_input=="exit":
        break
  response = chat_model.invoke(chat_history)
  chat_history.append(AIMessage(content=response.content))
  print("AI Generated Answer:",response.content)



In [ ]:
print("/n")